In [12]:
import pandas as pd
import os
import numpy as np
import numpy.ma as ma
import glob
from netCDF4 import Dataset,date2num,num2date
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xlsxwriter

In [13]:
def QPE(Zh,ZDR,min_index_lat,min_index_lon,a,b,c):
    R_mm_h=a*np.power(Zh[min_index_lat,min_index_lon],b)*np.power(ZDR[min_index_lat,min_index_lon],c)
    return R_mm_h

In [22]:
#read informations of gauge stations
stations=pd.read_excel(pd.ExcelFile("C:/Users/Admin/OneDrive/Desktop/gauges2.xlsx"))
#Choose a Case: Embedded/Squall line/Full stratiform
#Testing-Full stratiform:1,7,8,9,12,14,15/Squall line:2,6,8,10,11,12/Embedded:1,2,3,7,9,10,12,13,14,16,17,18,19
Case='Embedded.xlsx'
#Choose events of a case
Case_dic={Case:[str(i) for i in[1,2,3,7,9,10]]}
frames=[]
#sheets=[]
for r in Case_dic[Case]:
    #make gauges dataframe at each events
    df_gauges=pd.read_excel('D:/gauge datas 2021/Testing cases/'+Case,sheet_name=r)
    #make dataframes for each dual-pol variables
    linkNC='E:/SVM_netCDF/'+Case[:-5]+'/'+r+'/'
    df_CS_SVM=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_R=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_Zh=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    df_ZDR=pd.DataFrame(0,columns=df_gauges.columns[3:],index=stations['Tên Trạm'])
    n=0
    for file_name in os.listdir(linkNC):
        data=Dataset(linkNC+file_name,'r') # at 1 time
        Zh=np.ma.power(10.,0.1*data.variables['Zh_1.5km'][0,:,:])
        ZDR=np.ma.power(10.,0.1*data.variables['ZDR_1.5km'][0,:,:])
        CS_SVM=data.variables['CS_SVM'][0,:,:].filled(fill_value=0.0)
        for index,row in stations.iterrows():
            locations=row['Tên Trạm']
            location_lat=row['Latitude']
            location_lon=row['Longitude']
            lat=data.variables['latitude'][:]
            lon=data.variables['longitude'][:]
            sq_diff_lat=(lat-location_lat)**2
            sq_diff_lon=(lon-location_lon)**2
            min_index_lat=sq_diff_lat.argmin()
            min_index_lon=sq_diff_lon.argmin()
            if CS_SVM[min_index_lat,min_index_lon] == 1:
                df_CS_SVM.loc[[locations],list(df_gauges.columns[3:])[n]]=1
                df_R.loc[[locations],list(df_gauges.columns[3:])[n]]=QPE(Zh,ZDR,min_index_lat,min_index_lon,a=1.25,b=0.24,c=-1.19)
            elif CS_SVM[min_index_lat,min_index_lon] == 2:
                df_CS_SVM.loc[[locations],list(df_gauges.columns[3:])[n]]=2
                df_R.loc[[locations],list(df_gauges.columns[3:])[n]]=QPE(Zh,ZDR,min_index_lat,min_index_lon,a=0.19,b=0.56,c=-1.65)
            else:
                df_CS_SVM.loc[[locations],list(df_gauges.columns[3:])[n]]=0
                df_R.loc[[locations],list(df_gauges.columns[3:])[n]]=0
            df_Zh.loc[[locations],list(df_gauges.columns[3:])[n]]=Zh[min_index_lat,min_index_lon]
            df_ZDR.loc[[locations],list(df_gauges.columns[3:])[n]]=ZDR[min_index_lat,min_index_lon]
        n=n+1
    df=pd.DataFrame(data={
    'Stations': list(df_gauges['Trạm Đo'][:])*df_gauges.columns[3:].size,
    'times': list(np.repeat(df_gauges.columns[3:],df_gauges['Trạm Đo'].size)),
    'SC_SVM':list(pd.Series(df_CS_SVM.iloc[:,:].values.ravel('F'))),
    'R_10p':list(pd.Series(df_gauges.iloc[:,3:].values.ravel('F'))*6),
    'R_pre':list(pd.Series(df_R.iloc[:,:].values.ravel('F'))),
    'Z_1500m':list(pd.Series(df_Zh.iloc[:,:].values.ravel('F'))),
    'ZDR_1500m':list(pd.Series(df_ZDR.iloc[:,:].values.ravel('F')))    
    })
    #df=df.fillna(0)
    frames.append(df)
    print(r)
result = pd.concat(frames,ignore_index=True)
df_result=result.loc[(result.R_10p>0)&(result.R_pre>0)]
#df_Convection=result.loc[(result.R_10p>0)&(result.SC_SVM==2)]
zz=[df_result]
sheets=['QPE']
out_path = r"D:/QPE_evaluation/"+Case
writer = pd.ExcelWriter(out_path,engine='xlsxwriter')   
for dataframe, sheet in zip(zz, sheets):
    dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0,index=False)   
writer.save()

1
2
3
7
9
10
